# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [2]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()
doj.shape

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

(13087, 6)

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [3]:
## your code to subset to one press release and take the string
pharma = str(doj[doj["id"] == "17-1204"]["contents"])
type(pharma)

str

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [4]:
## your code here to restrict to alpha
processed = ""

for a in pharma:
    if a.isalpha() or a == " ":
        processed += a

In [5]:
## your code here for part of speech tagging

tokens = word_tokenize(processed)
token_pos = pos_tag(tokens)
token_pos

[('The', 'DT'),
 ('founder', 'NN'),
 ('and', 'CC'),
 ('majority', 'NN'),
 ('owner', 'NN'),
 ('of', 'IN'),
 ('Insys', 'NNP'),
 ('Therapeutics', 'NNP'),
 ('Inc', 'NNP'),
 ('was', 'VBD'),
 ('arrested', 'VBN'),
 ('today', 'NN'),
 ('and', 'CC'),
 ('charged', 'VBN'),
 ('with', 'IN'),
 ('leading', 'VBG'),
 ('a', 'DT'),
 ('nationwide', 'JJ'),
 ('conspiracy', 'NN'),
 ('to', 'TO'),
 ('profit', 'VB'),
 ('by', 'IN'),
 ('using', 'VBG'),
 ('bribes', 'NNS'),
 ('and', 'CC'),
 ('fraud', 'NN'),
 ('to', 'TO'),
 ('cause', 'VB'),
 ('the', 'DT'),
 ('illegal', 'JJ'),
 ('distribution', 'NN'),
 ('of', 'IN'),
 ('a', 'DT'),
 ('Fentanyl', 'NNP'),
 ('spray', 'NN'),
 ('intended', 'VBD'),
 ('for', 'IN'),
 ('cancer', 'NN'),
 ('patients', 'NNS'),
 ('experiencing', 'VBG'),
 ('breakthrough', 'IN'),
 ('painMore', 'NN'),
 ('than', 'IN'),
 ('Americans', 'NNPS'),
 ('died', 'VBD'),
 ('of', 'IN'),
 ('synthetic', 'JJ'),
 ('opioid', 'NN'),
 ('overdoses', 'NNS'),
 ('last', 'JJ'),
 ('year', 'NN'),
 ('and', 'CC'),
 ('millions', 'N

## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [6]:
## your code here for part A
spacy_doc = nlp(pharma)
named = [token for token in spacy_doc.ents]

In [7]:
## your code here for part B
for token in named:
    if token.label_ == "LAW":
        print(token.text)

RICO
the Controlled Substances Act
RICO


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

In [8]:
## your code here 
# RICO is the Racketeer Influenced and Corrupt Organizations Act which targets illegal business operations, which can include 
# a pharmacy making false/unnecessary prescriptions for kickbacks, payment to a healthcare group for prescribing a certain medication.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [9]:
## your code here

years = [a for a in named if a.label_ == "DATE" and re.search("year", a.text)]
print(years)

[last year, 20 years, three years, five years, three years]


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [10]:
## your code here

s = "|"
keys = s.join([yr.text for yr in years])
spans = []
sections = []

for x in re.finditer(keys, pharma):
    spans.append(x.span())

for span in spans:
    sections.append(pharma[span[0] - 250:span[1] + 250])

sections
# The CEO could face up to 45 years in prison and 9 years probation (20 years 3 supervised for conspiracy to commit RICO,
# 20 years 3 supervised for conspiracy to commit mail and write fraud, 5 year 3 supervised for conspiracy to violate Anti-Kickback Law.

['ith leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accounta',
 ' our veterans and their families,” said Donna L. Neves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.\xa0 The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater tha

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [11]:
## your code here for subsetting
doj_subset = doj[(doj["topics_clean"] == "Civil Rights") |(doj["topics_clean"] == "Hate Crimes") | (doj["topics_clean"] == "Project Safe Childhood")]

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [12]:
## your code here to define function
def sentiment(release):
    spacy_doc = nlp(release)
    named = [re.escape(token.text) for token in spacy_doc.ents]
    j = "|"
    named = j.join(named)
    release = re.sub(named, "", release)
    sent_obj = SentimentIntensityAnalyzer()
    return sent_obj.polarity_scores(release)

In [13]:
## your code here executing the function
doj_subset["contents"].apply(sentiment)

77       {'neg': 0.199, 'neu': 0.751, 'pos': 0.049, 'compound': -0.9931}
155      {'neg': 0.133, 'neu': 0.799, 'pos': 0.068, 'compound': -0.9325}
157       {'neg': 0.093, 'neu': 0.83, 'pos': 0.077, 'compound': -0.7579}
162      {'neg': 0.126, 'neu': 0.789, 'pos': 0.085, 'compound': -0.9037}
168      {'neg': 0.178, 'neu': 0.778, 'pos': 0.044, 'compound': -0.9864}
                                      ...                               
13002     {'neg': 0.155, 'neu': 0.78, 'pos': 0.064, 'compound': -0.9689}
13032     {'neg': 0.082, 'neu': 0.813, 'pos': 0.105, 'compound': 0.7003}
13034    {'neg': 0.158, 'neu': 0.754, 'pos': 0.088, 'compound': -0.9648}
13068    {'neg': 0.139, 'neu': 0.762, 'pos': 0.099, 'compound': -0.9798}
13081    {'neg': 0.151, 'neu': 0.819, 'pos': 0.031, 'compound': -0.9921}
Name: contents, Length: 717, dtype: object

C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [14]:
## your code here
doj_subset_wscore = doj_subset.copy()
doj_subset_wscore["sentiment"] = doj_subset_wscore["contents"].apply(sentiment)

In [15]:
doj_subset_wscore["neg"] = doj_subset_wscore.sentiment.apply(lambda s: s["neg"])
doj_subset_wscore["neu"] = doj_subset_wscore.sentiment.apply(lambda s: s["neu"])
doj_subset_wscore["pos"] = doj_subset_wscore.sentiment.apply(lambda s: s["pos"])
doj_subset_wscore["compound"] = doj_subset_wscore.sentiment.apply(lambda s: s["compound"])

doj_subset_wscore.sort_values(by="neg", ascending=False).head(2)[["id","contents","neg"]]

,id,contents,neg
329,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.323
572,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.303


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [16]:
## agg and find the mean compound score by topic
doj_subset_wscore.groupby("topics_clean").agg({
    "compound" : "mean"
})

# the variation in sentiment scores could be because releases on hate crimes tend to have very negative sentiment and include words such as
# hate, racist, homophobic, while Project Safe Childhood have less of these highly negative words and Civil Rights has the least.

,compound
topics_clean,
Civil Rights,-0.093931
Hate Crimes,-0.930943
Project Safe Childhood,-0.681391


# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [17]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [18]:
# ### TESTING ONLY 

# ## your code defining a text processing function

# # First, add custom stopwords 
# list_stopwords = stopwords.words("english")
# list_stopwords_new = list_stopwords + custom_doj_stopwords

# def preprocess(single_string): 

#     # # Step 1: check for floats
#     # if not isinstance(single_string, str):
#     #     return None

#     # Converts the words to lowercase
#     string_lower = single_string.lower() 

#     # Removes stopwords, adding the custom stopwords 
#     # in your code cell below to the default stopwords list
#     nostop_string = [word for word in wordpunct_tokenize(string_lower)
#                      if word not in list_stopwords_new]

#     ## Uses the snowball stemmer from nltk to stem
#     snow = SnowballStemmer(language='english')

#     ## Only retains alpha words (so removes digits and punctuation)
#     ## And only retains words 4 characters or longer
#     result_listing_preprocess = [snow.stem(token) 
#                             for token in nostop_string
#                             if token.isalpha() and 
#                             len(token) > 3]

#     ## Returns a joined preprocessed string    
#     result = " ".join(nostop_string)

#     return result 

In [19]:
# your code here to define the function(s)

# # First, add custom stopwords 
list_stopwords = stopwords.words("english")
list_stopwords_new = list_stopwords + custom_doj_stopwords

def preprocess(string):

    # Step 1: check for floats
    if not isinstance(string, str):
        return None
    
    ## Step 2: remove stopwords
    cleaned_listing = [word for word in wordpunct_tokenize(string.lower()) 
                          if word not in list_stopwords_new]

    
    snow = SnowballStemmer(language='english')
    
    ## Step 3: stem 
    result_listing = [snow.stem(token) for token in cleaned_listing if token.isalpha() and len(token) > 3]

    result = ", ".join(result_listing)

    return result

# your code here to apply the function

def preprocess_all(df):
    results = [] # list of lists 
    
    for x in df:
        preprocess_res = preprocess(x)
        results.append(preprocess_res)

    return results 

# For all texts in corpus_lower 
doj_subset_wscore["processed_text"] = preprocess_all(doj_subset_wscore.contents)
doj_subset_wscore

,id,title,contents,date,topics_clean,components_clean,sentiment,neg,neu,pos,compound,processed_text
77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle","{'neg': 0.199, 'neu': 0.751, 'pos': 0.049, 'compound': -0.9931}",0.199,0.751,0.049,-0.9931,"former, supervisori, correct, offic, louisiana, state, penitentiari, angola, louisiana, plead, guilti, yesterday, connect, beat, handcuf, shackl, inmat, addit, conspir, cover, misconduct, falsifi, offici, record, lie, intern, investig, happen, jame, savoy, marksvill, louisiana, admit, plea, hear, wit, offic, use, excess, forc, inmat, fail, interven, conspir, offic, cover, beat, engag, varieti, obstruct, act, person, falsifi, offici, prison, record, cover, attack, scotti, kennedi, beeb, arkansa, john, sander, marksvill, louisiana, previous, plead, guilti, novemb, septemb, role, beat, cover, everi, citizen, right, process, protect, unreason, forc, correct, offic, violat, basic, constitut, must, held, account, egregi, action, said, act, general, john, gore, continu, vigor, prosecut, correct, offic, violat, public, trust, commit, crime, cover, violat, feder, crimin, yesterday, anoth, exampl, unwav, commit, pursu, violat, feder, crimin, law, said, act, unit, state, middl, louisiana, corey, amundson, continu, work, close, ensur, investig, baton, roug, resid, agenc, prosecut, frederick, menner, middl, louisiana, christoph, perra, crimin, section"
155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence intr

In [20]:
# ## your code executing the function

# ## Use apply or list comprehension to execute that function
# ## and create a new column in the data called processed_text

# def preprocess_all(df):
#     results = [] # list of lists 
    
#     for x in df:
#         preprocess_res = preprocess(x)
#         results.append(preprocess_res)

#     return results 

# # For all texts in corpus_lower 
# # doj_subset_wscore["processed_text"] = doj_subset_wscore.contents.apply(preprocess_all)
# # doj_subset_wscore

# doj_subset_wscore["processed_text"] = doj_subset_wscore.contents.apply(preprocess)

In [21]:
## your code showing the examples

# Print the id, contents, and processed_text columns for the following press releases:

cols_of_interest = ["id", "contents", "processed_text"]

# id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

doj_subset_wscore[cols_of_interest][doj_subset_wscore.id == "16-718"]

# id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)

doj_subset_wscore[cols_of_interest][doj_subset_wscore.id == "16-217"]


,id,contents,processed_text
11593,16-718,"In a nine-count indictment unsealed today, two Mississippi correctional officers were charged with beating an inmate and a third was charged with helping to cover it up. The indictment charged Lawardrick Marsher, 28, and Robert Sturdivant, 47, officers at Mississippi State Penitentiary, in Parchman, Mississippi, with a beating that included kicking, punching and throwing the victim to the ground. Marsher and Sturdivant were charged with violating the right of K.H., a convicted prisoner, to be free from cruel and unusual punishment. Sturdivant was also charged with failing to intervene while Marsher was punching and beating K.H. The indictment alleges that their actions involved the use of a dangerous weapon and resulted in bodily injury to the victim. A third officer, Deonte Pate, 23, was charged along with Marsher and Sturdivant for conspiring to cover up the beating. The indictment alleges that all three officers submitted false reports and that all three lied to the FBI. If convicted, Marsher and Sturdivant face a maximum sentence of 10 years in prison on the excessive force charges. Each of the three officers faces up to five years in prison on the conspiracy and false statement charges, and up to 20 years in prison on the false report charges. An indictment is merely an accusation, and the defendants are presumed innocent unless and until proven guilty. This case is being investigated by the FBI’s Jackson Division, with the cooperation of the Mississippi Department of Corrections. It is being prosecuted by Assistant U.S. Attorney Robert Coleman of the Northern District of Mississippi and Trial Attorney Dana Mulhauser of the Civil Rights Division’s Criminal Section. Marsher Indictment","nine, count, indict, unseal, today, mississippi, correct, offic, charg, beat, inmat, third, charg, help, cover, indict, charg, lawardrick, marsher, robert, sturdiv, offic, mississippi, state, penitentiari, parchman, mississippi, beat, includ, kick, punch, throw, victim, ground, marsher, sturdiv, charg, violat, right, convict, prison, free, cruel, unusu, punish, sturdiv, also, charg, fail, interven, marsher, punch, beat, indict, alleg, action, involv, danger, weapon, result, bodili, injuri, victim, third, offic, deont, pate, charg, along, marsher, sturdiv, conspir, cover, beat, indict, alleg, three, offic, submit, fals, report, three, lie, convict, marsher, sturdiv, face, maximum, sentenc, year, prison, excess, forc, charg, three, offic, face, five, year, prison, conspiraci, fals, statement, charg, year, prison, fals, report, charg, indict, mere, accus, defend, presum, innoc, unless, proven, guilti, investig, jackson, cooper, mississippi, correct, prosecut, robert, coleman, northern, mississippi, dana, mulhaus, crimin, section, marsher, indict"


,id,contents,processed_text
6727,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a renewed commitment by the city of Miami and Chief Rodolfo Llanes to provide constitutional policing for Miami residents and to protect public safety through sustainable reform,” said Principal Deputy Assistant Attorney General Gupta. “The agreement will help to strengthen the relationship between the MPD and the communities they serve by improving accountability for officers who fire their weapons unlawfully, and provides for community participation in the enforcement of this agreement.” “Today's agreement is the result of a joint effort between the Department of Justice and the City of Miami to ensure that the Miami Police Department continues its efforts to make our community safe while protecting the sacred Constitutional rights of all of our citizens,” said U.S. Attorney Ferrer. “Through oversight and communication, the agreement seeks to make permanent the positive changes that former Chief Orosa and Chief Llanes have made, and we applaud the City Commission’s vote.” The settlement agreement builds upon important reforms implemented by the city since the Justice Department issued its findings, including: The investigation was conducted by attorneys and staff from the Civil Rights Division’s Special Litigation Section and the Civil Division of the U. S. Attorney’s Office of the Southern District of Florida.","reach, comprehens, settlement, agreement, citi, miami, miami, polic, resolv, offic, involv, shoot, offic, announc, princip, deputi, general, vanita, gupta, head, wifredo, ferrer, southern, florida, settlement, approv, miami, citi, commiss, today, effect, agreement, sign, parti, resolv, claim, stem, offic, involv, shoot, offic, conduct, violent, crime, control, enforc, find, issu, juli, identifi, pattern, practic, excess, forc, offic, involv, shoot, violat, fourth, amend, constitut, citi, complianc, settlement, monitor, independ, review, former, tampa, florida, polic, chief, jane, castor, settlement, agreement, citi, implement, comprehens, reform, ensur, constitut, polic, support, public, trust, settlement, agreement, design, minim, offic, involv, shoot, effect, quick, investig, offic, involv, shoot, occur, measur, includ, settlement, repres, renew, commit, citi, miami, chief, rodolfo, llane, provid, constitut, polic, miami, resid, protect, public, safeti, sustain, reform, said, princip, deputi, general, gupta, agreement, help, strengthen, relationship, communiti, serv, improv, account, offic, fire, weapon, unlaw, provid, communiti, part

## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [22]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

# need to ask get_feature_names_out 


## function provided
# def create_dtm(list_of_strings, metadata):
#     """ 
#     Function to create dense document-term matrix (DTM) from a list of strings and provided metadata. 
#     A sparse DTM is a list of term_index/doc_index tuples: if a given term occurs in a given doc at least once, 
#         then this count is listed as a tuple; if not, that term/doc pair is omitted. 
#     In a dense DTM, each row is one text (e.g., an Airbnb listing), each column is a term, and 
#         each cell indicates the frequency of that word in that text. 
    
#     Parameters:
#         list_of_strings (Series): each row contains a preprocessed string (need not be tokenized)
#         metadata (DataFrame): contains document-level covariates
    
#     Returns:
#         Dense DTM with metadata on left and then one column per word in lexicon
#     """
    
#     # initialize a sklearn tokenizer; this helps us tokenize the preprocessed string input
#     vectorizer = CountVectorizer(lowercase = True) 
#     dtm_sparse = vectorizer.fit_transform(list_of_strings)
#     print('Sparse matrix form:\n', dtm_sparse[:3]) # take a look at sparse representation
#     print()
    
#     # switch the dataframe from the sparse representation to the normal dense representation (so we can treat it as regular dataframe)
#     dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), columns=vectorizer.get_feature_names_out ())
#     print('Dense matrix form:\n', dtm_dense_named.head()) # take a look at dense representation
#     dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1) # add back document-level covariates

#     return(dtm_dense_named_withid)

doj_subset_wscore["compound"] = doj_subset_wscore.sentiment.apply(lambda s: s["compound"])
doj_subset_wscore

,id,title,contents,date,topics_clean,components_clean,sentiment,neg,neu,pos,compound,processed_text
77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle","{'neg': 0.199, 'neu': 0.751, 'pos': 0.049, 'compound': -0.9931}",0.199,0.751,0.049,-0.9931,"former, supervisori, correct, offic, louisiana, state, penitentiari, angola, louisiana, plead, guilti, yesterday, connect, beat, handcuf, shackl, inmat, addit, conspir, cover, misconduct, falsifi, offici, record, lie, intern, investig, happen, jame, savoy, marksvill, louisiana, admit, plea, hear, wit, offic, use, excess, forc, inmat, fail, interven, conspir, offic, cover, beat, engag, varieti, obstruct, act, person, falsifi, offici, prison, record, cover, attack, scotti, kennedi, beeb, arkansa, john, sander, marksvill, louisiana, previous, plead, guilti, novemb, septemb, role, beat, cover, everi, citizen, right, process, protect, unreason, forc, correct, offic, violat, basic, constitut, must, held, account, egregi, action, said, act, general, john, gore, continu, vigor, prosecut, correct, offic, violat, public, trust, commit, crime, cover, violat, feder, crimin, yesterday, anoth, exampl, unwav, commit, pursu, violat, feder, crimin, law, said, act, unit, state, middl, louisiana, corey, amundson, continu, work, close, ensur, investig, baton, roug, resid, agenc, prosecut, frederick, menner, middl, louisiana, christoph, perra, crimin, section"
155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence intr

In [23]:
# your code here

# A. Use the create_dtm function I provide
# and create a document-term matrix using the preprocessed press releases;
# make sure metadata contains the following columns: id, compound sentiment 
# column you added, and the topics_clean column

# example application on raw lowercase texts; 
dtm_pre = create_dtm(list_of_strings = doj_subset_wscore.processed_text,
                      metadata = doj_subset_wscore[['id', 'compound', 'topics_clean']])

# dtm_nopre

# ## show first set of rows/cols
# dtm_pre.head()

# ## show arbitrary later cols in resulting data
# dtm_nopre.shape
# dtm_nopre.iloc[0:5, 480:500]

# ## summing each col
# top_terms = dtm_nopre[dtm_nopre.columns[4:]].sum(axis = 0)

# ## sorting from most frequent to least frequent
# top_terms.sort_values(ascending = False)

# top_terms = dtm_pre[dtm_pre.columns[4:]].sum(axis = 0)

#top_terms = dtm_pre.iloc[:, 3:].sum(axis=0)

# result = top_terms.sort_values(ascending = False).iloc[0:4]

# print(result)

def get_topwords(quantile):

    level = dtm_pre.compound.quantile(quantile)  

    new_dtm = dtm_pre[dtm_pre.compound >= level]

    # top_terms = dtm_pre[dtm_pre.columns[3:]].sum(axis = 0)
    top_terms = new_dtm.iloc[:, 3:].sum(axis=0)

    result = top_terms.sort_values(ascending = False).iloc[0:10]
    
    return result 

# B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)
get_topwords(0.95)

# C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)
#doj_subset_wscore.apply(get_topwords(0.05))

# D. Print the top 10 words for press releases in each of the three topics_clean

ValueError: cannot reindex on an axis with duplicate labels

## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [ ]:
# your code here

# 3.1


## Step 1: re-tokenize and store in list
## here, i'm doing with the raw random sample of text
## in activity, you should do with the preprocessed texts
text_raw_tokens = [wordpunct_tokenize(one_text) for one_text in 
                  doj_subset_wscore.processed_text]


## Step 2: use gensim create dictionary - gets all unique words across documents
text_raw_dict = corpora.Dictionary(text_raw_tokens)
raw_len = len(text_raw_dict) # get length for comparison below

### explore first few keys and values
### see that key is just an arbitrary counter; value is the word itself
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]}


## Step 3: filter out very rare and very common words
## here, i'm using the threshold that a word needs to appear in at least
## 5% of docs but not more than 95%
## this is an integer count of docs so i round
lower_bound = round(doj_subset_wscore.shape[0]*0.05)
upper_bound = round(doj_subset_wscore.shape[0]*0.95)

### apply filtering to dictionary
text_raw_dict.filter_extremes(no_below = lower_bound,
                             no_above = upper_bound)
print(f'Filtering out very rare and very common words reduced the \
length of dictionary from {str(raw_len)} to {str(len(text_raw_dict))}.')
{k: text_raw_dict[k] for k in list(text_raw_dict)[:5]} # show first five entries after filtering


## Step 4: apply dictionary to TOKENIZED texts
## this creates a mapping between each word 
## in a specific listing and the key in the dictionary.
## for words that remain in the filtered dictionary,
## output is a list where len(list) == n documents
## and each element in the list is a list of tuples
## containing the mappings
corpus_fromdict = [text_raw_dict.doc2bow(one_text) 
                   for one_text in text_raw_tokens]

### can apply doc2bow(one_text, return_missing = True) to print words
### eliminated from the listing bc they're not in filtered dictionary.
### but feeding that one with missing values to
### the lda function can cause errors
corpus_fromdict_showmiss = [text_raw_dict.doc2bow(one_text, return_missing = True)
                            for one_text in text_raw_tokens]
print('Sample of documents represented in dictionary format (with omitted words noted):')
corpus_fromdict_showmiss[:10]


In [ ]:
# 3.2

## Step 5: we're finally ready to estimate the model!
## full documentation here - https://radimrehurek.com/gensim/models/ldamodel.html
## here, we're feeding the lda function:
## (1) the corpus we created from the dictionary,
## (2) a parameter we decide on for the number of topics (k),
## (3) the dictionary itself,
## (4) parameter for number of passes through training data (more means slower), and
## (5) parameter that returns, for each word remaining in dict, the topic probabilities.
## see documentation for many other arguments you can vary
ldamod = gensim.models.ldamodel.LdaModel(corpus_fromdict, 
                                         num_topics = 3, 
                                         id2word=text_raw_dict, 
                                         passes=6, 
                                         alpha = 'auto',
                                         per_word_topics = True,
                                         random_state = 6) # random_state

print(type(ldamod))


In [ ]:
## Post-model 1: explore corpus-wide summary of topics
### getting the topics and top words; can retrieve diff top words
topics = ldamod.print_topics(num_words = 15)
for topic in topics:
    print(topic)

## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [ ]:
## your code here to get doc-level topic probabilities 

## Post-model 2: explore topics associated with each document
### for each item in our original dictionary, get list of topic probabilities
l=[ldamod.get_document_topics(item) for item in corpus_fromdict]
### print result
text_raw_tokens[0:15]
#l[0:5]

In [ ]:
## your code here to add those topic probabilities to the dataframe

In [ ]:
## your code here to summarize the topic proportions for each of the topics_clean 

# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [ ]:
## your code here

## toy example 

sm = doj_subset_wscore.processed_text.iloc[0:2].reset_index()

sm

def create_bigram_onedoc(processed_text):

    # First, split into words 
    words_split = [word.strip() for word in processed_text.split(',')]
    
    # print(words_split)

    result = []
    
    for first, second in zip(words_split, words_split[1:]):
        res = first + "_" + second
        result.append(res)

    result = ', '.join(result) # separate by comma

    return result

example_str = "depart, reach, settlem"
create_bigram_onedoc(example_str)

sm["test"] = sm.processed_text.apply(create_bigram_onedoc)
sm


In [ ]:
## A - Using the doj_subset_wscore data and the processed_text column (so the words after stemming/other preprocessing), 
# create a column in the data called processed_text_bigrams

doj_subset_wscore["processed_text_bigrams"] = doj_subset_wscore.processed_text.apply(create_bigram_onedoc)

doj_subset_wscore

In [ ]:
## B - Print the id, processed_text, and processed_text_bigram columns for press release with id = 16-21

## your code showing the examples

# Print the id, contents, and processed_text columns for the following press releases:

cols_of_interest_3 = ["id", "processed_text", "processed_text_bigrams"]

# id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)

doj_subset_wscore[cols_of_interest_3][doj_subset_wscore.id == "16-217"]


C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [ ]:
# C

dtm_bigram = create_dtm(list_of_strings=doj_subset_wscore.processed_text_bigrams,
                     metadata=doj_subset_wscore[['id', 'compound', 'topics_clean']])


In [ ]:
# D

dtm_pre.shape
dtm_pre
# (717, 10979)

dtm_bigram.shape
dtm_bigram
# (717, 72721)

# The bigram matrix has more columns than the unigram matrix because each pairing of words is taken into account, rather than
# each word as a unique entry. By keeping track of the relationships between two unique words, the number of columns 
# and unique words therefore is multiplied. 


In [ ]:
# E Find and print the 10 most prevelant bigrams for each of the three topics_clean using the get_topwords function from 2.2

print(get_topwords(bottom_5_percent_negative))

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [24]:
# your code here

# https://medium.com/wetheitguys/python-jellyfish-for-enhanced-string-matching-6fe18ee32c8b
# Note that I use jaccard similarity for computational savings! 

## extra imports
# !pip install jellyfish # can install on terminal or by uncommenting this line

import jellyfish

# Compare two strings
string1 = "qss20iscool"
string2 = "qss20isawesome"

# Calculate jaccard_similarity on a few test cases 
distance = jellyfish.jaccard_similarity(string1, string2)
print(f"Distance between '{string1}' and '{string2}' is: {distance}")
distance = jellyfish.jaccard_similarity(string2, string2)
print(f"Distance between '{string2}' and '{string2}' is: {distance}") # 1 means identical, so we want to sort by ascending values later on

# Preprocess doj_subset: 
doj_subset["processed_text"] = preprocess_all(doj_subset.contents)
doj_subset

Distance between 'qss20iscool' and 'qss20isawesome' is: 0.0
Distance between 'qss20isawesome' and 'qss20isawesome' is: 1.0


,id,title,contents,date,topics_clean,components_clean,processed_text
77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle","former, supervisori, correct, offic, louisiana, state, penitentiari, angola, louisiana, plead, guilti, yesterday, connect, beat, handcuf, shackl, inmat, addit, conspir, cover, misconduct, falsifi, offici, record, lie, intern, investig, happen, jame, savoy, marksvill, louisiana, admit, plea, hear, wit, offic, use, excess, forc, inmat, fail, interven, conspir, offic, cover, beat, engag, varieti, obstruct, act, person, falsifi, offici, prison, record, cover, attack, scotti, kennedi, beeb, arkansa, john, sander, marksvill, louisiana, previous, plead, guilti, novemb, septemb, role, beat, cover, everi, citizen, right, process, protect, unreason, forc, correct, offic, violat, basic, constitut, must, held, account, egregi, action, said, act, general, john, gore, continu, vigor, prosecut, correct, offic, violat, public, trust, commit, crime, cover, violat, feder, crimin, yesterday, anoth, exampl, unwav, commit, pursu, violat, feder, crimin, law, said, act, unit, state, middl, louisiana, corey, amundson, continu, work, close, ensur, investig, baton, roug, resid, agenc, prosecut, frederick, menner, middl, louisiana, christoph, perra, crimin, section"
155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in G

In [28]:
# Now, use a loop: 

from itertools import combinations

result = []

# levenshtein_distance was too computationally expensive

for string1, string2 in combinations(range(len(doj_subset)), 2):
    distance = jellyfish.jaccard_similarity(doj_subset["processed_text"].iloc[string1], doj_subset["processed_text"].iloc[string2])
    result.append((string1, string2, distance))


In [26]:
# Next, verify results:  

len(result)

# result

256686

In [27]:
# Finally, put results into a dataframe, and sort values!

res_df = pd.DataFrame(result, columns=["string1", "string2", 'jaccard_sim'])

# Find the top two pairs (so four press releases total) 
res_df.sort_values(by="jaccard_sim", ascending=False).iloc[0:2]

# Most similar pair 
doj_subset.contents.iloc[465]
doj_subset.contents.iloc[466]

# This seems like the same press release was used to describe the sentencing of two different individuals for the same crime 
# (and the same type and length of punishment, as seen in 210 months in prison and 10 years of supervised release).
# So in this first case, it was just press releases covering similar crimes.

# Second most similar pair 
doj_subset.contents.iloc[203]
doj_subset.contents.iloc[686]

# However, in this second case, they seem like different stages of the same crime. For the same individual,
# press release 686 details his guilty plea (conviction), while press release 203 details his sentencing 
# to 20 years in prison to be followed by a lifetime term of supervised release.

,string1,string2,jaccard_sim
225060,465,466,0.938889
125327,203,686,0.916168


'A Wichita, Kansas man was sentenced to 210 months in prison and 10 years of supervised release, for production of child pornography based on his participation in a website that was operated for the purpose of coercing and enticing minors as young as eight years old to engage in sexually explicit conduct on web camera. \xa0 Acting Assistant Attorney General Kenneth A. Blanco of the Justice Department’s Criminal Division; U.S. Attorney Dana J. Boente of the Eastern District of Virginia; and Section Chief John J. Brosnan of the FBI’s Violent Crimes Against Children Section (VCACS) made the announcement. \xa0 Allan Cortez, 34, was charged on April 4, 2016, and pleaded guilty before U.S. District Judge T.S. Ellis III of the Eastern District of Virginia on April 14. \xa0 According to admissions made in connection with the plea agreement, members of the conspiracy created false profiles on social networking sites popular with children, posing as young teenagers to lure children to two websit

'A Chicopee, Massachusetts man was sentenced to 210 months in prison and 10 years of supervised release for production of child pornography based on his participation in a website that was operated for the purpose of coercing and enticing minors as young as eight years old to engage in sexually explicit conduct on web camera.\xa0 Acting Assistant Attorney General Kenneth A. Blanco of the Justice Department’s Criminal Division; U.S. Attorney Dana J. Boente of the Eastern District of Virginia; and Section Chief John J. Brosnan of the FBI’s Violent Crimes Against Children Section (VCACS) made the announcement. Edward Parson, 46, was charged on April 4, 2016, and pleaded guilty before U.S. District Judge T.S. Ellis III of the Eastern District of Virginia on April 14.\xa0 According to admissions made in connection with the plea agreement, members of the conspiracy created false profiles on social networking sites popular with children, posing as young teenagers to lure children to two websi

'A Church Hill, Maryland, resident was sentenced today to 20 years in prison to be followed by a lifetime term of supervised release for enticement of a minor to engage in sexual activity and attempting to transfer obscene materials to a minor, announced Acting Assistant Attorney General Kenneth A. Blanco of the Justice Department’s Criminal Division and Acting U.S. Attorney Benjamin G. Greenberg of the Southern District of Florida.  \xa0 Lee Robert Moore, 38, pleaded guilty March 1, 2017, before U.S. District Judge Daniel T. K. Hurley of the Southern District of Florida. Moore was employed by the U.S. Secret Service-Uniformed Division and was assigned to the White House at the time of his arrest on Nov. 9, 2015, and has remained in custody since that time. Moore has since been terminated from his Secret Service position. \xa0 According to admissions made in connection with his plea, Moore maintained a profile on the social media application “Meet24,” which provides a mobile-based plat

'A Church Hill, Maryland resident pleaded guilty today in federal court to one count of enticement of a minor to engage in sexual activity and one count of attempting to transfer obscene materials to a minor, announced Acting Assistant Attorney General Kenneth A. Blanco of the Justice Department’s Criminal Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida.\xa0\xa0 Lee Robert Moore, 38, pleaded guilty today before U.S. District Judge Daniel T. K. Hurley of the Southern District of Florida.\xa0\xa0Moore was employed by the U.S. Secret Service-Uniformed Division and was assigned to the White House at the time of his arrest on Nov. 9, 2015, and has remained in custody since that time.\xa0\xa0Moore has since been terminated from his Secret Service position. \xa0 According to admissions made in connection with his plea, Moore maintained a profile on the social media application “Meet24,” which provides a mobile-based platform for exchanging digital images, as w